In [1]:
import os, io, json, urllib, numpy as np, codecs, os.path,time

In [ ]:
def get_data(cursor,i,year):
    global item, all_items, url
    item={}; items=[]; 
    p={ 'filter':'from-pub-date:'+year+',until-pub-date:'+year , 'rows' : '1000', 'cursor' :cursor}
    parameters=urllib.urlencode(p)
    url='http://api.crossref.org/types/journal-article/works?'+parameters
    response = urllib.urlopen(url).read()
    #if response=='Resource not found.': return
    json_result=json.loads(response)
    items = json_result['message']['items']
    if len(items)>0:
        f=codecs.open('data/'+year+'/'+str(i)+'.txt','a','utf-8')
        for item in items:
            author='0';year_print=9999;year_online=9999; doi=issn=journal=pub=cite=ref=issnp=issne=lic=''
            if 'published-print' in item:
                year_print=item['published-print']['date-parts'][0][0]
            if 'published-online' in item:
                year_online=item['published-online']['date-parts'][0][0]
            year=str(min(year_print,year_online))
            if 'DOI' in item: doi=item['DOI']
            if 'issn-type' in item: 
                for i in item['issn-type']:
                    if i['type']=='print': 
                        issnp=i['value']
                    elif i['type']=='electronic':
                        issne=i['value']
            if 'container-title' in item: journal=','.join(item['container-title'])
            if 'publisher' in item: pub=item['publisher']; pub=''if pub==None else pub
            if 'is-referenced-by-count' in item: cite=str(item['is-referenced-by-count'])
            if 'references-count' in item: ref=str(item['references-count'])
            if 'author' in item: author='1'
            if 'license' in item: 
                lic=item['license'][0]['URL'].split('://')[1].split('/')[0]
                #print lic
            res= doi+'\t'+year+'\t'+ref+'\t'+cite+'\t'+issnp+'\t'+issne+'\t'+journal+'\t'+pub+'\t'+author+'\t'+lic+'\n'
            f.write(res)    
        f.close()
    next_cursor=json_result['message']['next-cursor']
    return next_cursor

In [ ]:
for year in range(2015,2018):
    year=str(year)
    if not os.path.exists('data/'+year):
        os.makedirs('data/'+year)    
    cursor=''
    next_cursor='*'
    item={}
    all_items=[]
    f=open('log/'+year+'.txt','a')
    i=0
    while next_cursor!=cursor:
        cursor=next_cursor
        f.write(str(i)+' , '+cursor+'\n')
        while True:
            try:
                next_cursor=get_data(cursor,i,year)
            except Exception,e: 
                print str(e)
                time.sleep(1)
                continue
            break
        i=i+1
        if i%10==0:print i,

10

In [ ]:
url

In [ ]:
cursor